In [10]:
from domain import Intent

# Cas valide
intent = Intent(
    type="get_goals",
    team1="Chelsea",
    team2="Burnley",
    match_date="2015-02-21"
)

intent.validate()  # ne doit rien lever
print("OK")


OK


In [11]:
# test_data.py
from pathlib import Path

from data import SoccerNetDataClient


def run_smoke_test():
    # 1) Ton root dataset (à adapter)
    ROOT = Path(r"C:\Users\Ilias\Documents\cours_MINES\TR\projet\data\data")

    # 2) Instanciation
    client = SoccerNetDataClient(ROOT)
    print("[OK] DataClient initialisé")

    # 3) Test résolution match_id (exemple Chelsea vs Burnley)
    match_id = client.resolve_match_id(
        team1="Manchester United",
        team2="Swansea",
        match_date="2015-02-21",
        competition="england_epl",
        season="2014-2015",
    )
    print("[OK] match_id résolu:", match_id)

    # 4) Test chargement des deux mi-temps
    halves = client.load_match(match_id)
    assert len(halves) == 2, "load_match doit renvoyer 2 mi-temps"

    h1, h2 = halves
    assert h1.half == 1 and h2.half == 2, "Mauvaises mi-temps"
    assert h1.step_seconds == 0.5 and h2.step_seconds == 0.5, "step_seconds attendu = 0.5"

    # 5) Test shape embeddings
    assert hasattr(h1.embeddings, "shape"), "embeddings doit avoir un attribut shape"
    assert len(h1.embeddings.shape) == 2, "embeddings doit être de shape (T, D)"
    print("[OK] H1 embeddings shape:", h1.embeddings.shape)
    print("[OK] H2 embeddings shape:", h2.embeddings.shape)

    # 6) Test conversion index -> temps
    t = 10
    time_sec = h1.index_to_time_sec(t)
    assert abs(time_sec - 5.0) < 1e-9, "Avec step_seconds=0.5, index 10 doit donner 5.0s"
    print("[OK] index_to_time_sec(10) =", time_sec)

    print("\n✅ SMOKE TEST data.py PASSED")


if __name__ == "__main__":
    run_smoke_test()


[OK] DataClient initialisé
[OK] match_id résolu: england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United
[OK] H1 embeddings shape: (5450, 512)
[OK] H2 embeddings shape: (5874, 512)
[OK] index_to_time_sec(10) = 5.0

✅ SMOKE TEST data.py PASSED


In [14]:
import numpy as np
from domain import HalfEmbeddings
from models import TemporalPostProcessor

# fake half embeddings (juste pour step_seconds + match_id)
half = HalfEmbeddings(match_id="dummy", half=1, embeddings=np.zeros((2000, 512)), step_seconds=0.5)

# fake probas: un pic autour de t=1000
probs = np.random.rand(2000) * 0.1
probs[1000] = 0.95
probs[1005] = 0.90  # devrait être supprimé par min_separation_seconds si trop proche

pp = TemporalPostProcessor(threshold=0.5, min_separation_seconds=10.0)
events = pp.extract_events_for_label(probs, half, label="Goal")

print(events)
print(events[0].mmss(), events[0].time_sec)


[ActionEvent(match_id='dummy', half=1, time_sec=500.0, label='Goal', confidence=0.95, source='transformer', extra={'t_index': 1000})]
08:20 500.0
